<a href="https://colab.research.google.com/github/tomonari-masada/course2023-nlp/blob/main/assignment20230923.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 課題20230923
* TF-IDFベクトルを使って、20 newsgroupsの任意のテキストについて、それと最も似ているテキストを10個返す関数を書こう。
* 10個のうち、元のテキストと同じクラスに属するテキストがいくつあるかを調べよう。
* 全てのテキストについて同じことを行ない、最も似ている上位10個のうち同じクラスのテキスト数の平均値を求めよう。

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups

newsgroups = fetch_20newsgroups()

In [6]:
def number_normalizer(tokens):
  return ("#NUMBER" if token[0].isdigit() else token for token in tokens)

class NumberNormalizingVectorizer(TfidfVectorizer):
  def build_tokenizer(self):
    tokenizer = super().build_tokenizer()
    return lambda doc: list(number_normalizer(tokenizer(doc)))

In [7]:
vectorizer = NumberNormalizingVectorizer(stop_words='english', min_df=5)
X = vectorizer.fit_transform(newsgroups.data).toarray()
X.shape

(11314, 23427)

In [ ]:
def find_similar_texts(text_idx, num_texts=10):
  return np.argsort(- np.dot(X, X[text_idx]))[:num_texts]

In [8]:
num_texts = 10

text_idx = 0
similar_texts = find_similar_texts(text_idx, num_texts=num_texts)
(newsgroups.target[similar_texts] == newsgroups.target[text_idx]).sum()

5

In [10]:
from tqdm import tqdm

num_correct_answers = list()
for text_idx in tqdm(range(len(newsgroups.target))):
  similar_texts = find_similar_texts(text_idx, num_texts=num_texts)
  num_correct_answers.append(
      (newsgroups.target[similar_texts] == newsgroups.target[text_idx]).sum()
      )

  1%|          | 82/11314 [00:16<38:06,  4.91it/s]


KeyboardInterrupt: ignored

* これだと遅いので、改良。

In [13]:
batch_size = 1000

def find_similar_texts(text_idx, num_texts=10):
  return np.argsort(- X[text_idx:text_idx+batch_size] @ X.T, axis=-1)[:,:num_texts]

In [14]:
num_texts = 10

text_idx = 0
similar_texts = find_similar_texts(text_idx, num_texts=num_texts)
similar_texts.shape

(1000, 10)

In [20]:
(newsgroups.target[similar_texts] == newsgroups.target[text_idx:text_idx+batch_size].reshape(-1,1)).sum()

6698

* これで大丈夫そう。

In [21]:
from tqdm import tqdm

num_correct_answers = list()
for text_idx in tqdm(range(0, len(newsgroups.target)+1, 1000)):
  similar_texts = find_similar_texts(text_idx, num_texts=num_texts)
  num_correct_answers.append(
      (newsgroups.target[similar_texts] == newsgroups.target[text_idx:text_idx+batch_size].reshape(-1,1)).sum()
      )

100%|██████████| 12/12 [03:59<00:00, 19.93s/it]


In [22]:
np.array(num_correct_answers).sum()

75611

In [23]:
np.array(num_correct_answers).sum() / len(newsgroups.target)

6.682959165635496